In [1]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")

import json

In [2]:
df = pd.read_excel('../data/Data_2020_P06/all_billings_inputs_08_20.xlsx',
                  sheet_name = 'base_billings')

In [3]:
print('df length: ', len(df))

df length:  294044


In [4]:
df.rename(
        index=str,
        columns={
            "Contrct Duration in Months": "duration",
            "Document Currency": "curr",
            "Enterprise BU Desc": "BU",
            "Invoice Fiscal Year Period Desc": "period",
            "POB Type": "POB_type",
            "Product Config Type": "config",
            "Rev Rec Category": "rev_req_type",
            "Rule For Bill Date": "rebill_rule",
            "Sales Document Type": "sales_doc",
            "Sales Type": "sales_type",
            "Subscription Term": "sub_term",
            "Completed Sales ( DC )": "DC_amount",
            "Completed Sales": "US_amount",
        },
        inplace=True,
    )


In [5]:
df.head(3)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.0,278.01
1,0,AUD,Experience Cloud,2017-01,NaN,ONORE,NaN,NaN,ZCC,RECOGNIZED,0,0.0,45.75
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.0,-512177.81


In [6]:
print(df.sales_doc.value_counts())


ZCSB    153000
ZCC      31544
ZLO3     25077
ZCCR     19879
ZLO5     15356
ZLG2     10547
ZCQ       7574
ZLO1      5908
ZCDR      3325
ZLDR      2328
ZRL2      2172
ZLO2      2032
ZOG2      2022
ZLCR      1912
OR        1863
ZCOR      1530
KE        1427
ZCRE      1071
ZLO4      1016
ZOTM       840
G2         813
KR         540
ZRG2       526
ZATS       461
RE         380
ZCS1       193
ZPS        159
ZREB       136
ZOL2       122
ZACR        98
KL          97
ZCKL        45
ZCSR        14
ZCP1        11
ZCPR         9
ZCS2         7
L2           6
ZCQA         3
ZLKL         1
Name: sales_doc, dtype: int64


In [7]:
vc = df["curr"].value_counts()
vc

USD    74655
EUR    46083
GBP    32417
AUD    30933
JPY    26693
SEK    16383
CHF    16160
DKK    15277
NOK    14704
HKD     3081
BRL     2965
CAD     2787
RUB     2634
KRW     1176
CLP      948
ARS      936
COP      920
INR      761
SGD      755
PEN      741
TWD      568
THB      528
PHP      501
MYR      494
IDR      485
ILS      137
TRY      121
NZD       59
SAR       29
KWD       22
BHD       20
JOD       19
EGP       17
OMR       16
LBP       15
BMD        2
AED        1
MXP        1
Name: curr, dtype: int64

In [8]:
vc.values

array([74655, 46083, 32417, 30933, 26693, 16383, 16160, 15277, 14704,
        3081,  2965,  2787,  2634,  1176,   948,   936,   920,   761,
         755,   741,   568,   528,   501,   494,   485,   137,   121,
          59,    29,    22,    20,    19,    17,    16,    15,     2,
           1,     1])

In [9]:
keep_these = vc.values > 20
print(keep_these)
keep_curr = vc[keep_these]
keep_curr

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False False]


USD    74655
EUR    46083
GBP    32417
AUD    30933
JPY    26693
SEK    16383
CHF    16160
DKK    15277
NOK    14704
HKD     3081
BRL     2965
CAD     2787
RUB     2634
KRW     1176
CLP      948
ARS      936
COP      920
INR      761
SGD      755
PEN      741
TWD      568
THB      528
PHP      501
MYR      494
IDR      485
ILS      137
TRY      121
NZD       59
SAR       29
KWD       22
Name: curr, dtype: int64

In [10]:
list_keepers = keep_curr.index
remove_these = vc[vc.values <= 20].index
print(remove_these)

Index(['BHD', 'JOD', 'EGP', 'OMR', 'LBP', 'BMD', 'AED', 'MXP'], dtype='object')


In [11]:
print(type(vc))
print(type(remove_these))

<class 'pandas.core.series.Series'>
<class 'pandas.core.indexes.base.Index'>


In [12]:
print(list_keepers)

Index(['USD', 'EUR', 'GBP', 'AUD', 'JPY', 'SEK', 'CHF', 'DKK', 'NOK', 'HKD',
       'BRL', 'CAD', 'RUB', 'KRW', 'CLP', 'ARS', 'COP', 'INR', 'SGD', 'PEN',
       'TWD', 'THB', 'PHP', 'MYR', 'IDR', 'ILS', 'TRY', 'NZD', 'SAR', 'KWD'],
      dtype='object')


In [13]:
model_dict = {"curr_removed": list(vc[remove_these].index)}
model_dict

{'curr_removed': ['BHD', 'JOD', 'EGP', 'OMR', 'LBP', 'BMD', 'AED', 'MXP']}

In [14]:
delete_curr = list(remove_these)
print(delete_curr)

['BHD', 'JOD', 'EGP', 'OMR', 'LBP', 'BMD', 'AED', 'MXP']


In [15]:
if "TRY" not in model_dict["curr_removed"]:
    model_dict["curr_removed"].append("TRY")
    delete_curr.append("TRY")
    list_keepers = list_keepers.drop("TRY")

df = df[df["curr"].isin(list_keepers)]

In [16]:
df.head(20)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.00,278.01
1,0,AUD,Experience Cloud,2017-01,NaN,ONORE,NaN,NaN,ZCC,RECOGNIZED,0,0.00,45.75
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.00,-512177.81
3,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,16196.47,11935.54
4,0,AUD,Experience Cloud,2017-01,NaN,OUNIV,B,NaN,ZCC,DEFERRED,0,-38000.00,-27070.00
5,0,CAD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,25171.20,18816.73
6,0,EUR,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,525.00,552.92
7,0,EUR,Experience Cloud,2017-01,NaN,OCONS,B,NaN,ZCC,DEFERRED,0,-3790.50,-4464.85
8,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YA,ZCC,DEFERRED,0,17256.00,18352.45
9,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YC,ZCC,DEFERRED,0,-17256.00,-17982.82


In [17]:
print(len(df))
df = df[df["DC_amount"] != 0]
print(len(df))

293832
286744


In [18]:
model_dict

{'curr_removed': ['BHD',
  'JOD',
  'EGP',
  'OMR',
  'LBP',
  'BMD',
  'AED',
  'MXP',
  'TRY']}

In [19]:
df['POB_type'].value_counts(dropna=False)

NaN      192219
RR        73256
IR-NA      7424
IR         4819
BNDL       3604
CR         2956
RR-NA      2308
CR-NA       102
LFB          56
Name: POB_type, dtype: int64

## Grouping by POB Type

The new 606 has everything grouped by sales type. There are blanks
We need to separate into groups based on the POB type


In [27]:
list_IR = ['IR', 'IR-NA', 'LFB']
list_service = ['CR', 'CR-NA']
list_deferred = ['RR', 'RR-NA']
list_hybrid = ['BNDL']

list_all = list_IR + list_service + list_deferred + list_hybrid
print(list_all)

['IR', 'IR-NA', 'LFB', 'CR', 'CR-NA', 'RR', 'RR-NA', 'BNDL']


In [28]:
rec = df[df["POB_type"].isin(list_IR)].copy()
svc = df[df["POB_type"].isin(list_service)].copy()
dfr = df[df["POB_type"].isin(list_deferred)].copy()
hyb = df[df["POB_type"].isin(list_hybrid)].copy()
df_no_POB = df[~df["POB_type"].isin(list_all)].copy()

In [29]:
print('rec', len(rec))
print('svc', len(svc))
print('dfr', len(dfr))
print('hyb', len(hyb))
print('no_POB', len(df_no_POB))
print('ALL', len(df))

rec 12299
svc 3058
dfr 75564
hyb 3604
no_POB 192219
ALL 286744


In [30]:
df_no_POB.tail(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
294034,96,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,930.83,930.83
294035,97,DKK,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,370.00,54.02
294036,97,EUR,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,249.95,271.09
294037,97,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,34080.00,317.46
294038,97,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,1094.79,1094.79
294039,98,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,5680.00,52.77
294040,98,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,208.96,208.96
294041,99,AUD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,139.98,90.40
294042,99,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,-5680.00,-52.26
294043,99,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,211.96,211.96


In [32]:
# Blank
df_no_POB.head(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.00,278.01
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.00,-512177.81
3,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,16196.47,11935.54
4,0,AUD,Experience Cloud,2017-01,NaN,OUNIV,B,NaN,ZCC,DEFERRED,0,-38000.00,-27070.00
5,0,CAD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,25171.20,18816.73
6,0,EUR,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,525.00,552.92
7,0,EUR,Experience Cloud,2017-01,NaN,OCONS,B,NaN,ZCC,DEFERRED,0,-3790.50,-4464.85
8,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YA,ZCC,DEFERRED,0,17256.00,18352.45
9,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YC,ZCC,DEFERRED,0,-17256.00,-17982.82
10,0,EUR,Experience Cloud,2017-01,NaN,ONORE,D,Y1,ZCC,DEFERRED,0,-13868.36,-15248.40


In [33]:
blank.tail(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
294034,96,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,930.83,930.83
294035,97,DKK,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,370.00,54.02
294036,97,EUR,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,249.95,271.09
294037,97,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,34080.00,317.46
294038,97,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,1094.79,1094.79
294039,98,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,5680.00,52.77
294040,98,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,208.96,208.96
294041,99,AUD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,139.98,90.40
294042,99,JPY,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,-5680.00,-52.26
294043,99,USD,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,211.96,211.96


#### TODO:
 - create function that moves through these by rev_rec_class and classifies them based on if they are A, B, D or F
 - classify them as in A and D
 - return  dataframes
     -IR
     -service
     -deferred
 - merge these with the other dataframes
 - Move on to type A

In [36]:
df_no_POB['sales_doc'].value_counts(dropna=False)

ZCSB    152438
ZCC       7293
ZCCR      6987
ZLO3      5420
ZLG2      3325
ZLO5      3169
ZLO1      1957
ZCQ       1767
ZCDR      1745
ZCOR       954
ZLDR       906
ZRL2       826
ZOG2       734
ZLCR       696
OR         653
ZLO2       648
ZCRE       529
G2         418
ZOTM       404
KE         271
ZATS       213
KR         202
ZRG2       195
RE         149
ZREB       136
ZACR        56
ZCS1        43
ZOL2        39
ZPS         32
ZCSR         4
L2           4
ZCQA         3
ZCS2         3
Name: sales_doc, dtype: int64

In [37]:
sales_doc_type= {
        "immediate_revenue": [
            "L2",
            "RE",
            "G2",
            "KE",
            "KR",
            "OR",
            "ZACR",
            "ZCCR",
            "ZCDR",
            "ZCOR",
            "ZCSR",
            "ZCRE",
            "ZCS2",
            "ZLO3",
            "ZLO1",
            "ZLO2",
            "ZOG2",
            "ZOL2",
            "ZOTM"
        ],
        "service": [
            "ZATS",
            "ZPS"
        ],
        "deferred": [
            "ZCC",
            "ZCS1",
            "ZCQ",
            "ZCQA",
            "ZLCR",
            "ZLDR",
            "ZLG2",
            "ZLO5",
            "ZREB",
            "ZRG2",
            "ZRL2",
            "ZCSB"
        ]
    }
    

In [38]:
sales_doc_type

{'immediate_revenue': ['L2',
  'RE',
  'G2',
  'KE',
  'KR',
  'OR',
  'ZACR',
  'ZCCR',
  'ZCDR',
  'ZCOR',
  'ZCSR',
  'ZCRE',
  'ZCS2',
  'ZLO3',
  'ZLO1',
  'ZLO2',
  'ZOG2',
  'ZOL2',
  'ZOTM'],
 'service': ['ZATS', 'ZPS'],
 'deferred': ['ZCC',
  'ZCS1',
  'ZCQ',
  'ZCQA',
  'ZLCR',
  'ZLDR',
  'ZLG2',
  'ZLO5',
  'ZREB',
  'ZRG2',
  'ZRL2',
  'ZCSB']}

In [39]:
rec_no_POB = df_no_POB[df_no_POB['sales_doc'].isin(sales_doc_type['immediate_revenue'])].copy()
svc_no_POB = df_no_POB[df_no_POB['sales_doc'].isin(sales_doc_type['service'])].copy()
dfr_no_POB = df_no_POB[df_no_POB['sales_doc'].isin(sales_doc_type['deferred'])].copy()


In [40]:
print(len(blank))
print('rec ', len(rec_no_POB))
print('svc ', len(svc_no_POB))
print('def ', len(dfr_no_POB))

print((len(blank) - len(rec_no_POB) - len(svc_no_POB) - len(dfr_no_POB)))

192219
rec  21177
svc  245
def  170797
0


In [41]:
dfr_no_POB.sample(20)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
227384,15,USD,Creative,2019-11,NaN,1Y,D,YA,ZCSB,DEFERRED,12,-7957.96,-7957.96
292685,53,GBP,Creative,2020-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,858.19,1058.56
261858,19,JPY,Creative,2020-03,NaN,MTHLY,D,Y3,ZCSB,DEFERRED,1,1651479.00,15037.28
154442,50,DKK,Creative,2019-01,NaN,1Y,D,YA,ZCSB,DEFERRED,12,6720.00,1021.04
182872,40,SEK,Creative,2019-05,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,10384.80,1120.11
100781,3,AUD,Creative,2018-06,NaN,1Y,A,NaN,ZLG2,DEFERRED,12,-584.51,-439.26
186816,9,USD,Document Cloud,2019-06,NaN,MTHLY,D,Y3,ZCSB,DEFERRED,1,12059.15,12059.15
39987,8,DKK,Creative,2017-08,NaN,MTHLY,D,Y3,ZCSB,DEFERRED,1,12650.00,1942.58
189515,27,SEK,Creative,2019-06,NaN,1Y,D,Y3,ZCSB,DEFERRED,1,22072.86,2311.22
258312,4,HKD,Document Cloud,2020-03,NaN,NaN,D,Y3,ZCSB,DEFERRED,1,1399.07,180.07


In [42]:
# pass in df_no_POB
def classify_no_POB(config_dict, df):
    
    df.drop(labels=['duration'], axis=1, inplace=True)
    # first group based on rev rec type
    
    df_a = df[df["rev_req_type"] == "A"].copy()
    gb_a = df_a.groupby(["curr", "BU", "period", "config", "sub_term"], as_index=False).sum()

    config_type_keepers = config_dict['type_A_config_keepers']                                  
    gb_a_keepers = gb_a[gb_a["config"].isin(config_type_keepers)].copy()
    df_a_no_config = gb_a[~gb_a["config"].isin(config_type_keepers)].copy()
    
    gb_a_1M = gb_a_keepers[(gb_a_keepers['config'] == 'MTHLY') |
                         (gb_a_keepers['sub_term'] == 1)].copy()
    index_1M = gb_a_1M.index
    # dropping monthly billings from the keepers
    gb_a_keepers.drop(index_1M, inplace=True)

    gb_a_1Y = gb_a_keepers[(gb_a_keepers['sub_term'] == 12) |
                         ((gb_a_keepers['sub_term'] == 0) &
                          (gb_a_keepers['config'] == '1Y'))].copy()
    index_1Y = gb_a_1Y.index
    gb_a_keepers.drop(index_1Y, inplace=True)

    # remaining types will be 2Y with sub_term = 0 or 24 or 3Y with 0 or 36
    gb_a_2Y = gb_a_keepers[gb_a_keepers['config'] == '2Y'].copy()
    gb_a_3Y = gb_a_keepers[gb_a_keepers['config'] == '3Y'].copy()

    print("this is the length of type A 1M billings: ", len(gb_a_1M))
    print("this is the length of type A 1Y billings: ", len(gb_a_1Y))
    print("this is the length of type A 2Y billings: ", len(gb_a_2Y))
    print("this is the length of type A 3Y billings: ", len(gb_a_3Y))

    # Cleaning up the sub_term columns from the gb_A_#X variabels
    gb_a_1M.drop(labels = ['sub_term'], axis=1, inplace=True)
    gb_a_1Y.drop(labels=['sub_term'], axis=1, inplace=True)
    gb_a_2Y.drop(labels=['sub_term'], axis=1, inplace=True)
    gb_a_3Y.drop(labels=['sub_term'], axis=1, inplace=True)
    # dropping duration from the gb_a_#X below here

    # NOW I AM WORKING ON TYPE D
    df_d = df[df["rev_req_type"] == "D"].copy()

    gb_d = df_d.groupby(["curr", "BU", "period", "rebill_rule", "sales_doc"], as_index=False).sum()
    gb_d.drop(labels=["sub_term"], axis=1, inplace=True)

    gb_d["rebill_rule"].value_counts(dropna=False)

    list_monthly = config_dict['type_D_classification']["list_monthly"]
    list_qtrly = config_dict['type_D_classification']["list_qtrly"]
    list_semi_ann = config_dict['type_D_classification']["list_semi_ann"]
    list_ann = config_dict['type_D_classification']["list_ann"]
    list_2yrs = config_dict['type_D_classification']["list_2yrs"]
    list_3yrs = config_dict['type_D_classification']["list_3yrs"]
    list_all_rebills = list_monthly + list_qtrly + list_semi_ann + list_ann + list_2yrs + list_3yrs

    
    gb_d_mthly = gb_d[gb_d["rebill_rule"].isin(list_monthly)].copy()
    gb_d_mthly.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_mthly = gb_d_mthly.groupby(["curr", "BU", "period"]).sum()
    gb_d_mthly.reset_index(inplace=True)

    gb_d_qtrly = gb_d[gb_d["rebill_rule"].isin(list_qtrly)].copy()
    gb_d_qtrly.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_qtrly = gb_d_qtrly.groupby(["curr", "BU", "period"]).sum()
    gb_d_qtrly.reset_index(inplace=True)

    gb_d_semi_ann = gb_d[gb_d["rebill_rule"].isin(list_semi_ann)]
    gb_d_semi_ann.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_semi_ann = gb_d_semi_ann.groupby(["curr", "BU", "period"]).sum()
    gb_d_semi_ann.reset_index(inplace=True)

    gb_d_annual = gb_d[gb_d["rebill_rule"].isin(list_ann)].copy()
    gb_d_annual.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_annual = gb_d_annual.groupby(["curr", "BU", "period"]).sum()
    gb_d_annual.reset_index(inplace=True)

    gb_d_two_yrs = gb_d[gb_d["rebill_rule"].isin(list_2yrs)].copy()
    gb_d_two_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_two_yrs = gb_d_two_yrs.groupby(["curr", "BU", "period"]).sum()
    gb_d_two_yrs.reset_index(inplace=True)

    gb_d_three_yrs = gb_d[gb_d["rebill_rule"].isin(list_3yrs)]
    gb_d_three_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_three_yrs = gb_d_three_yrs.groupby(["curr", "BU", "period"]).sum()
    gb_d_three_yrs.reset_index(inplace=True)

    # TODO: Create a check to see if there are any type D deferred billings with no rebill rule
    gb_d_no_rebill = gb_d[~gb_d['rebill_rule'].isin(list_all_rebills)].copy()
    print('There are {} line items that are type D and have no rebill rule'.format(len(gb_d_no_rebill)))

    print("Length of monthly", len(gb_d_mthly))
    print("Length of quarterly", len(gb_d_qtrly))
    print("Length of semi ann", len(gb_d_semi_ann))
    print("Length of annual", len(gb_d_annual))
    print("Length of two years", len(gb_d_two_yrs))
    print("Length of three years", len(gb_d_three_yrs))
    
    


In [121]:
def process_type_A(config_dict, df):
    '''
    Type A Billings

    These billings are on a billing plan. The product config tells us how long before they renew
    and the sub term determines how often they are billed. (See the document "Deferred Revenue Model August 2020"
    in the docs subfolder for a better understanding of the details.

    1M: config = 'MTHLY' or sub_term = 1
    1Y: config = '1Y' AND sub term = 0 or 12  OR any config with sub_term = 12
    2Y: config = '2Y' AND sub term = 0 or 24
    3Y: config = '3Y' AND sub term = 0 or 36
    There are also config types that do not allow us to map these into a billings frequency.
    These types are {"BLANK", "OCONS", "ONORE", "OUNIV"}
        These types are loaded from the type_A_no_config report, but we pass them back to the main
        program to test that we have not dropped anything.

    :param config_dict: This is the json config file. It contains a list 'type_A_config_keepers' that
            has the list of config types we know how to process (i.e. we know how they will be rebilled)

    :param df: This is the dataframe of deferred billings that have revenue recognition type A
    :return: return_A_dict: A dictionary containing dataframes containing the billings that have the same
                rebill frequency.
                return_A_dict = {'gb_a_1M': gb_a_1M,
                     'gb_a_1Y': gb_a_1Y,
                     'gb_a_2Y': gb_a_2Y,
                     'gb_a_3Y': gb_a_3Y,
                     'gb_a_no_config': gb_a_no_config,}
    '''

    # grouping by fields we need to keep.
    gb_a = df.groupby(["curr", "BU", "period", "config", "sub_term"], as_index=False).sum()

    print('A config value counts')
    print(gb_a["config"].value_counts(dropna=False))

    # splitting into config types we keep and ones we need to get in the type_A_no_config report
    config_type_keepers = config_dict['type_A_config_keepers']
    gb_a_keepers = gb_a[gb_a["config"].isin(config_type_keepers)].copy()
    gb_a_no_config = gb_a[~gb_a["config"].isin(config_type_keepers)].copy()

    print('len gb_a', len(gb_a))
    print('gb_a_keepers', len(gb_a_keepers))
    print('len df_a_no_config', len(gb_a_no_config))
    print('Total USD Equivalent Billings of Type A with bad configs', gb_a_no_config.US_amount.sum())

    # ###### Grouping by the config type into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M dataframes
    # Selecting monthly billings
    gb_a_1M = gb_a_keepers[(gb_a_keepers['config'] == 'MTHLY') |
                           (gb_a_keepers['sub_term'] == 1)].copy()
    index_1M = gb_a_1M.index
    # dropping monthly billings from the keepers
    gb_a_keepers.drop(index_1M, inplace=True)

    gb_a_1Y = gb_a_keepers[(gb_a_keepers['sub_term'] == 12) |
                           ((gb_a_keepers['sub_term'] == 0) &
                            (gb_a_keepers['config'] == '1Y'))].copy()
    index_1Y = gb_a_1Y.index
    gb_a_keepers.drop(index_1Y, inplace=True)

    # remaining types will be 2Y with sub_term = 0 or 24 or 3Y with 0 or 36
    gb_a_2Y = gb_a_keepers[gb_a_keepers['config'] == '2Y'].copy()
    gb_a_3Y = gb_a_keepers[gb_a_keepers['config'] == '3Y'].copy()

    print("this is the length of type A 1M billings: ", len(gb_a_1M))
    print("this is the length of type A 1Y billings: ", len(gb_a_1Y))
    print("this is the length of type A 2Y billings: ", len(gb_a_2Y))
    print("this is the length of type A 3Y billings: ", len(gb_a_3Y))

    # Cleaning up the sub_term columns from the gb_A_#X variabels
    gb_a_1M.drop(labels=['sub_term'], axis=1, inplace=True)
    gb_a_1Y.drop(labels=['sub_term'], axis=1, inplace=True)
    gb_a_2Y.drop(labels=['sub_term'], axis=1, inplace=True)
    gb_a_3Y.drop(labels=['sub_term'], axis=1, inplace=True)
    # dropping duration from the gb_a_#X below here
    return_A_dict = {'gb_a_1M': gb_a_1M,
                     'gb_a_1Y': gb_a_1Y,
                     'gb_a_2Y': gb_a_2Y,
                     'gb_a_3Y': gb_a_3Y,
                     'gb_a_no_config': gb_a_no_config,}
    return return_A_dict


In [43]:
def process_type_D(config_dict, df):
    '''
    TYPE D billings
    These billings have a field 'Rule For Bill Date' that determines when new billings will occur
     - Monthly:        *{Y1, Y2, Y3, Y5}*
     - Quarterly:      *YQ*
     - Every 4 months: *YT*  --NOTE: There are only 10 of these, so I am treating these as quarterly--
     - Semi-annual:    *YH*
     - Annual:         *{YA, YC}*
     - Every 2 years:  *Y4*
     - Every 3 years:  *Y7*

     We also need to track the type D billings that do not have a 'Rule for Bill Date'

    :param config_dict: This is the json config file. It contains a dictionary 'type_D_config_classification'
        that has the list of rebill_rules we know how to process (i.e. we know when the contract will end)

    :param df: This is the dataframe of deferred billings that have revenue recognition type D

    :return: return_D_dict: A dictionary containing dataframes containing the billings that have the same
                rebill_rule frequency.
                return_dict = {'monthly': gb_d_mthly,
                    'qtrly': gb_d_qtrly,
                   'semi_ann': gb_d_semi_ann,
                   'annual': gb_d_annual,
                   'two_years': gb_d_two_yrs,
                   'three_years': gb_d_three_yrs,
                   'no_rebill': gb_d_no_rebill}

    '''
    #  We also need to track the type D billings that do not have a 'Rule for Bill Date'
    gb_d = df.groupby(["curr", "BU", "period", "rebill_rule", "sales_doc"], as_index=False).sum()
    gb_d.drop(labels=["sub_term"], axis=1, inplace=True)

    gb_d["rebill_rule"].value_counts(dropna=False)

    # ###### Grouping these by rebill rule and incorporating rebill rules that have the same rebill period
    list_monthly = config_dict['type_D_classification']["list_monthly"]
    list_qtrly = config_dict['type_D_classification']["list_qtrly"]
    list_semi_ann = config_dict['type_D_classification']["list_semi_ann"]
    list_ann = config_dict['type_D_classification']["list_ann"]
    list_2yrs = config_dict['type_D_classification']["list_2yrs"]
    list_3yrs = config_dict['type_D_classification']["list_3yrs"]
    list_all_rebills = list_monthly + list_qtrly + list_semi_ann + list_ann + list_2yrs + list_3yrs

    gb_d_mthly = gb_d[gb_d["rebill_rule"].isin(list_monthly)].copy()
    gb_d_mthly.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_mthly = gb_d_mthly.groupby(["curr", "BU", "period"]).sum()
    gb_d_mthly.reset_index(inplace=True)

    gb_d_qtrly = gb_d[gb_d["rebill_rule"].isin(list_qtrly)].copy()
    gb_d_qtrly.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_qtrly = gb_d_qtrly.groupby(["curr", "BU", "period"]).sum()
    gb_d_qtrly.reset_index(inplace=True)

    gb_d_semi_ann = gb_d[gb_d["rebill_rule"].isin(list_semi_ann)]
    gb_d_semi_ann.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_semi_ann = gb_d_semi_ann.groupby(["curr", "BU", "period"]).sum()
    gb_d_semi_ann.reset_index(inplace=True)

    gb_d_annual = gb_d[gb_d["rebill_rule"].isin(list_ann)].copy()
    gb_d_annual.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_annual = gb_d_annual.groupby(["curr", "BU", "period"]).sum()
    gb_d_annual.reset_index(inplace=True)

    gb_d_two_yrs = gb_d[gb_d["rebill_rule"].isin(list_2yrs)].copy()
    gb_d_two_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_two_yrs = gb_d_two_yrs.groupby(["curr", "BU", "period"]).sum()
    gb_d_two_yrs.reset_index(inplace=True)

    gb_d_three_yrs = gb_d[gb_d["rebill_rule"].isin(list_3yrs)]
    gb_d_three_yrs.drop(labels="rebill_rule", axis=1, inplace=True)
    gb_d_three_yrs = gb_d_three_yrs.groupby(["curr", "BU", "period"]).sum()
    gb_d_three_yrs.reset_index(inplace=True)


    gb_d_no_rebill = gb_d[~gb_d['rebill_rule'].isin(list_all_rebills)].copy()

    print('There are {} line items that are type D and have no rebill rule'.format(len(gb_d_no_rebill)))
    print("Length of monthly", len(gb_d_mthly))
    print("Length of quarterly", len(gb_d_qtrly))
    print("Length of semi ann", len(gb_d_semi_ann))
    print("Length of annual", len(gb_d_annual))
    print("Length of two years", len(gb_d_two_yrs))
    print("Length of three years", len(gb_d_three_yrs))

    return_dict = {'monthly': gb_d_mthly,
                   'qtrly': gb_d_qtrly,
                   'semi_ann': gb_d_semi_ann,
                   'annual': gb_d_annual,
                   'two_years': gb_d_two_yrs,
                   'three_years': gb_d_three_yrs,
                   'no_rebill': gb_d_no_rebill}
    return return_dict


In [44]:
print(len(blank))
print('rec ', len(rec_no_POB))
print('svc ', len(svc_no_POB))
print('def ', len(dfr_no_POB))

print((len(blank) - len(rec_no_POB) - len(svc_no_POB) - len(dfr_no_POB)))

192219
rec  21177
svc  245
def  170797
0


In [45]:
dfr_no_POB['rev_req_type'].value_counts()

D    157047
A      7848
B       354
Name: rev_req_type, dtype: int64

In [46]:
df_A= dfr_no_POB[dfr_no_POB['rev_req_type']=='A'].copy()

In [47]:
df_A.tail(20)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
134755,3,USD,Document Cloud,2018-11,NaN,1Y,A,NaN,ZLG2,DEFERRED,12,-80.94,-80.94
134769,3,USD,DX Other,2018-11,NaN,NaN,A,NaN,ZRG2,DEFERRED,0,-270749.88,-270749.88
134770,3,USD,DX Other,2018-11,NaN,NaN,A,NaN,ZRL2,DEFERRED,0,69249.96,69249.96
134773,3,USD,Experience Cloud,2018-11,NaN,NaN,A,NaN,ZRL2,DEFERRED,0,150315.18,150315.18
134786,3,USD,LiveCycle,2018-11,NaN,NaN,A,NaN,ZRL2,DEFERRED,0,64698.78,64698.78
134787,3,USD,Print & Publishing,2018-11,NaN,NaN,A,NaN,ZRL2,DEFERRED,0,81594.43,81594.43
134788,3,USD,Print & Publishing,2018-11,NaN,1Y,A,NaN,ZLDR,DEFERRED,0,600.00,600.00
134789,3,USD,Print & Publishing,2018-11,NaN,1Y,A,NaN,ZLDR,DEFERRED,12,8625.00,8625.00
134987,5,EUR,Creative,2018-11,NaN,1Y,A,NaN,ZLCR,DEFERRED,12,-1324.60,-1527.35
135001,5,EUR,Document Cloud,2018-11,NaN,1Y,A,NaN,ZLCR,DEFERRED,12,-74.95,-86.42


In [49]:
df_B= dfr_no_POB[dfr_no_POB['rev_req_type']=='B'].copy()

In [50]:
df_B['US_amount'].sum()

-12577895.5

In [118]:
config_dict={
    "ADBE_cal": {
        "direct_filename": "../Data/OLD/ADOBE_FINANCIAL_CALENDAR.xlsx",
        "sheetname": "ADBE_cal"
    },
    "FX_forwards": {
        "filename": "FX_forward_rates.xlsx",
        "sheetname": "forward_data"
    },
    "FX_rates": {
        "filename": "FX_data.xlsx",
        "sheetname": "to_matlab"
    },
    "POB_type_classifier": {
        "list_IR": [
            "IR",
            "IR-NA",
            "LFB"
        ],
        "list_service": [
            "CR",
            "CR-NA"
        ],
        "list_deferred": [
            "RR",
            "RR-NA"
        ],
        "list_hybrid": [
            "BNDL"
        ]
    },
    "billings": {
        "filename": "all_billings_inputs_08_20.xlsx",
        "base_sheetname": "base_billings",
        "type_A_sheetname": "type_A_no_config"
    },
    "bookings": {
        "filename": "2020_bookings_fcst_Q2.xlsx",
        "sheetname": "source"
    },
    "curr_map": {
        "filename": "currency_map.xlsx",
        "sheetname": "curr_map"
    },
    "hybrid_pct_IR": 0.17,
    "list_columns": [
        "recognized_",
        "service_",
        "deferred_B_",
        "deferred_1M_",
        "deferred_3M_",
        "deferred_6M_",
        "deferred_1Y_",
        "deferred_2Y_",
        "deferred_3Y_"
    ],
    "list_future_periods": [
        "2020-07",
        "2020-08",
        "2020-09",
        "2020-10",
        "2020-11",
        "2020-12",
        "2021-01",
        "2021-02",
        "2021-03",
        "2021-04",
        "2021-05",
        "2021-06"
    ],
    "new_columns": [
        "fwd_01M",
        "fwd_02M",
        "fwd_03M",
        "fwd_04M",
        "fwd_05M",
        "fwd_06M",
        "fwd_07M",
        "fwd_08M",
        "fwd_09M",
        "fwd_10M",
        "fwd_11M",
        "fwd_12M"
    ],
    "path_to_data": "../data/Data_2020_P06/",
    "sales_doc_type": {
        "immediate_revenue": [
            "L2",
            "RE",
            "G2",
            "KE",
            "KR",
            "OR",
            "ZACR",
            "ZCCR",
            "ZCDR",
            "ZCOR",
            "ZCSR",
            "ZCRE",
            "ZCS2",
            "ZLO3",
            "ZLO1",
            "ZLO2",
            "ZOG2",
            "ZOL2",
            "ZOTM"
        ],
        "service": [
            "ZATS",
            "ZPS"
        ],
        "deferred": [
            "ZCC",
            "ZCS1",
            "ZCQ",
            "ZCQA",
            "ZLCR",
            "ZLDR",
            "ZLG2",
            "ZLO5",
            "ZREB",
            "ZRG2",
            "ZRL2",
            "ZCSB"
        ]
    },
    "type_A_config_keepers": [
        "MTHLY",
        "1Y",
        "2Y",
        "3Y"
    ],
    "type_D_classification": {
        "list_monthly": [
            "Y1",
            "Y2",
            "Y3",
            "YM"
        ],
        "list_qtrly": [
            "YQ",
            "YY",
            "YT"
        ],
        "list_semi_ann": [
            "YH"
        ],
        "list_ann": [
            "YA",
            "YC",
            "YX"
        ],
        "list_2yrs": [
            "Y4"
        ],
        "list_3yrs": [
            "Y7"
        ]
    }
}

In [122]:
A_dict = process_type_A(config_dict, df_A)

A config value counts
1Y       649
2Y        40
OUNIV     38
OCONS     31
MTHLY      2
Name: config, dtype: int64
len gb_a 760
gb_a_keepers 691
len df_a_no_config 69
Total USD Equivalent Billings of Type A with bad configs 1715354.5
this is the length of type A 1M billings:  60
this is the length of type A 1Y billings:  598
this is the length of type A 2Y billings:  33
this is the length of type A 3Y billings:  0


In [125]:
A_dict.keys()

dict_keys(['gb_a_1M', 'gb_a_1Y', 'gb_a_2Y', 'gb_a_3Y', 'gb_a_no_config'])

In [126]:
df_D= dfr_no_POB[dfr_no_POB['rev_req_type']=='D'].copy()

In [127]:
df_D.head(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.00,-512177.81
3,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,16196.47,11935.54
5,0,CAD,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,25171.20,18816.73
8,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YA,ZCC,DEFERRED,0,17256.00,18352.45
9,0,EUR,Experience Cloud,2017-01,NaN,OCONS,D,YC,ZCC,DEFERRED,0,-17256.00,-17982.82
10,0,EUR,Experience Cloud,2017-01,NaN,ONORE,D,Y1,ZCC,DEFERRED,0,-13868.36,-15248.40
11,0,EUR,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-411640.44,-440381.24
12,0,EUR,Experience Cloud,2017-01,NaN,ONORE,D,YQ,ZCC,DEFERRED,0,-392.08,-460.25
17,0,GBP,Experience Cloud,2017-01,NaN,OCONS,D,YA,ZCC,DEFERRED,0,16286.00,20461.08
19,0,GBP,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,184200.99,234614.89


In [130]:
df_D.rebill_rule.value_counts(dropna=False)

Y3     129001
YA      26070
YQ       1159
Y1        411
YH        144
YC         80
Y7         54
Y2         48
Y4         43
NaN        35
YM          1
YT          1
Name: rebill_rule, dtype: int64

In [111]:
df_B.config.value_counts()

OCONS    178
OUNIV     94
Name: config, dtype: int64

In [51]:
def classify_no_POB_ZCSB(config_dict, df):
    # does it have a rebill rule?, then calssify those as D
    list_monthly = config_dict['type_D_classification']["list_monthly"]
    list_qtrly = config_dict['type_D_classification']["list_qtrly"]
    list_semi_ann = config_dict['type_D_classification']["list_semi_ann"]
    list_ann = config_dict['type_D_classification']["list_ann"]
    list_2yrs = config_dict['type_D_classification']["list_2yrs"]
    list_3yrs = config_dict['type_D_classification']["list_3yrs"]
    list_all_rebills = list_monthly + list_qtrly + list_semi_ann + list_ann + list_2yrs + list_3yrs

    df_1M = df[df['rebill_rule'].isin(list_monthly)].copy()
    df_1M.drop(labels="rebill_rule", axis=1, inplace=True)
    df_1M = df_1M.groupby(["curr", "BU", "period"]).sum()
    df_1M.reset_index(inplace=True)

    df_1Q = df[df['rebill_rule'].isin(list_qtrly)].copy()
    df_1Q.drop(labels="rebill_rule", axis=1, inplace=True)
    df_1Q = df_1Q.groupby(["curr", "BU", "period"]).sum()
    df_1Q.reset_index(inplace=True)

    df_6M = df[df['rebill_rule'].isin(list_semi_ann)].copy()
    df_6M.drop(labels="rebill_rule", axis=1, inplace=True)
    df_6M = df_6M.groupby(["curr", "BU", "period"]).sum()
    df_6M.reset_index(inplace=True)

    df_1Y = df[df['rebill_rule'].isin(list_ann)].copy()
    df_1Y.drop(labels="rebill_rule", axis=1, inplace=True)
    df_1Y = df_1Y.groupby(["curr", "BU", "period"]).sum()
    df_1Y.reset_index(inplace=True)

    df_2Y = df[df['rebill_rule'].isin(list_2yrs)].copy()
    df_2Y.drop(labels="rebill_rule", axis=1, inplace=True)
    df_2Y = df_2Y.groupby(["curr", "BU", "period"]).sum()
    df_2Y.reset_index(inplace=True)

    df_3Y = df[df['rebill_rule'].isin(list_3yrs)].copy()
    df_3Y.drop(labels="rebill_rule", axis=1, inplace=True)
    df_3Y = df_3Y.groupby(["curr", "BU", "period"]).sum()
    df_3Y.reset_index(inplace=True)

    # which ones are blank
    df_ZCC_no_rebill = df[~df['rebill_rule'].isin(list_all_rebills)].copy()
    
    # classify based on sub term
    df_1M_sub_term = df_ZCC_no_rebill[df_ZCC_no_rebill['sub_term==1']].copy()
    df_1Y_sub_term = df_ZCC_no_rebill[df_ZCC_no_rebill['sub_term==12']].copy()
    df_other = df_ZCC_no_rebill[(df_ZCC_no_rebill['sub_term'!=1)&
                                                 (df_ZCC_no_rebill['sub_term']!=12])].copy()
                                 
    # if blank sub term - 
    
    ZCC_dict = {'df_1M': df_1M,
               'df_1Q': df_1Q,
               'df_6M': df_6M,
               'df_1Y': df_1Y,
                'df_2Y': df_2Y,
                'df_3Y': df_3Y,
                'df_1M_sub_term': df_1M_sub_term,
                'df_1Y_sub_term': df_1Y_sub_term
                'other': df_other
               }
                                
                                 
                                 
    return ZCC_dict                        
                                 

SyntaxError: invalid syntax (<ipython-input-51-5fb110c1bdba>, line 47)